# VSN Hackathon notebook

## Pre-requisites

The IAM role used to run this notebook should have below permissions

1. S3 access to create/upload objects
2. Bedrock full access
3. Custom policy for opensearch serverless

{
  "Version": "2012-10-17",
  "Statement": [
      {
          "Effect": "Allow",
          "Action": "aoss:*",
          "Resource": "*"
      }
  ]
}

In [1]:
# Ensure necessary packages are installed
%pip install -U opensearch-py==2.3.1
%pip install -U boto3==1.33.2
%pip install -U retrying==1.3.4
# Restart kernel

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import os
import boto3
from botocore.exceptions import ClientError
import pprint
from utility import create_bedrock_execution_role, create_oss_policy_attach_bedrock_execution_role, create_policies_in_oss, interactive_sleep
import random
from retrying import retry

sts_client = boto3.client('sts')
boto3_session = boto3.session.Session()
region_name = boto3_session.region_name
bedrock_agent_client = boto3_session.client('bedrock-agent', region_name=region_name)
service = 'aoss'
s3_client = boto3.client('s3')

In [3]:
#Create s3 bucket for storing Knowledge base

bucket_name = 'hackathon-vsn-bedrock-kb'
s3bucket = s3_client.create_bucket(Bucket=bucket_name)

## Create a vector store - OpenSearch Serverless index
## Step 1 - Create OSS policies and collection 


In [4]:
import boto3
import time
vector_store_name = 'hackathon-sample-rag'
index_name = 'hackathon-bedrock-sample-rag-index'
aoss_client = boto3_session.client('opensearchserverless')
bedrock_kb_execution_role = create_bedrock_execution_role(bucket_name=bucket_name)
bedrock_kb_execution_role_arn = bedrock_kb_execution_role['Role']['Arn']

In [5]:
# create security, network and data access policies within OSS
encryption_policy, network_policy, access_policy = create_policies_in_oss(vector_store_name=vector_store_name,
                       aoss_client=aoss_client,
                       bedrock_kb_execution_role_arn=bedrock_kb_execution_role_arn)
collection = aoss_client.create_collection(name=vector_store_name,type='VECTORSEARCH')

In [6]:
print(collection)

{'createCollectionDetail': {'arn': 'arn:aws:aoss:us-east-1:814180930505:collection/qu32cw1yb8mqdg0w0fpd', 'createdDate': 1728212750986, 'id': 'qu32cw1yb8mqdg0w0fpd', 'kmsKeyArn': 'auto', 'lastModifiedDate': 1728212750986, 'name': 'hackathon-sample-rag', 'standbyReplicas': 'ENABLED', 'status': 'CREATING', 'type': 'VECTORSEARCH'}, 'ResponseMetadata': {'RequestId': 'a440858c-2e5e-4f5a-86ad-5442357db68e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'a440858c-2e5e-4f5a-86ad-5442357db68e', 'date': 'Sun, 06 Oct 2024 11:05:51 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '312', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


In [7]:
# Get the OpenSearch serverless collection URL
collection_id = collection['createCollectionDetail']['id']
host = collection_id + '.' + region_name + '.aoss.amazonaws.com'
print(host)

qu32cw1yb8mqdg0w0fpd.us-east-1.aoss.amazonaws.com


In [9]:
# wait for collection creation
# This can take couple of minutes to finish
response = aoss_client.batch_get_collection(names=[vector_store_name])
print(response["collectionDetails"])

[{'arn': 'arn:aws:aoss:us-east-1:814180930505:collection/qu32cw1yb8mqdg0w0fpd', 'collectionEndpoint': 'https://qu32cw1yb8mqdg0w0fpd.us-east-1.aoss.amazonaws.com', 'createdDate': 1728212750986, 'dashboardEndpoint': 'https://qu32cw1yb8mqdg0w0fpd.us-east-1.aoss.amazonaws.com/_dashboards', 'id': 'qu32cw1yb8mqdg0w0fpd', 'kmsKeyArn': 'auto', 'lastModifiedDate': 1728212774797, 'name': 'hackathon-sample-rag', 'standbyReplicas': 'ENABLED', 'status': 'ACTIVE', 'type': 'VECTORSEARCH'}]


In [10]:
# create opensearch serverless access policy and attach it to Bedrock execution role
try:
    create_oss_policy_attach_bedrock_execution_role(collection_id=collection_id,
                                                    bedrock_kb_execution_role=bedrock_kb_execution_role)
    # It can take up to a minute for data access rules to be enforced
except Exception as e:
    print(e)

Opensearch serverless arn:  arn:aws:iam::814180930505:policy/AmazonBedrockOSSPolicyForKnowledgeBase_214


## Step 2 - Create vector index

In [11]:
# Create the vector index in Opensearch serverless, with the knn_vector field index mapping, specifying the dimension size, name and engine.
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth, RequestError
credentials = boto3.Session().get_credentials()
awsauth = auth = AWSV4SignerAuth(credentials, region_name, service)

index_name = 'hackathon-bedrock-sample-rag-index'
body_json = {
   "settings": {
      "index.knn": "true",
       "number_of_shards": 1,
       "knn.algo_param.ef_search": 512,
       "number_of_replicas": 0,
   },
   "mappings": {
      "properties": {
         "vector": {
            "type": "knn_vector",
            "dimension": 1536,
             "method": {
                 "name": "hnsw",
                 "engine": "faiss",
                 "space_type": "l2"
             },
         },
         "text": {
            "type": "text"
         },
         "text-metadata": {
            "type": "text"         }
      }
   }
}

# Build the OpenSearch client
oss_client = OpenSearch(
    hosts=[{'host': host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)


In [12]:
# Create index
try:
    response = oss_client.indices.create(index=index_name, body=json.dumps(body_json))
    print('\nCreating index:')
    print(response)

    # index creation can take up to a minute
except RequestError as e:
    # you can delete the index if its already exists
    # oss_client.indices.delete(index=index_name)
    print(f'Error while trying to create the index, with error {e.error}\nyou may unmark the delete above to delete, and recreate the index')
    


Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'hackathon-bedrock-sample-rag-index'}


## Create Knowledge Base
## Pre-requisite Upload data to s3

In [15]:


opensearchServerlessConfiguration = {
            "collectionArn": collection["createCollectionDetail"]['arn'],
            "vectorIndexName": index_name,
            "fieldMapping": {
                "vectorField": "vector",
                "textField": "text",
                "metadataField": "text-metadata"
            }
        }

# Ingest strategy - How to ingest data from the data source
chunkingStrategyConfiguration = {
    "chunkingStrategy": "FIXED_SIZE",
    "fixedSizeChunkingConfiguration": {
        "maxTokens": 512,
        "overlapPercentage": 20
    }
}

# The data source to ingest documents from, into the OpenSearch serverless knowledge base index
s3Configuration = {
    "bucketArn": f"arn:aws:s3:::{bucket_name}",
    # "inclusionPrefixes":["*.*"] # you can use this if you want to create a KB using data within s3 prefixes.
}

# The embedding model used by Bedrock to embed ingested documents, and realtime prompts
embeddingModelArn = f"arn:aws:bedrock:{region_name}::foundation-model/amazon.titan-embed-text-v1"

name = "hackathon-bedrock-sample-knowledge-base"
description = "Banta Claws"
roleArn = bedrock_kb_execution_role_arn


In [16]:
# Create a KnowledgeBase
from retrying import retry

@retry(wait_random_min=1000, wait_random_max=2000,stop_max_attempt_number=7)
def create_knowledge_base_func():
    create_kb_response = bedrock_agent_client.create_knowledge_base(
        name = name,
        description = description,
        roleArn = roleArn,
        knowledgeBaseConfiguration = {
            "type": "VECTOR",
            "vectorKnowledgeBaseConfiguration": {
                "embeddingModelArn": embeddingModelArn
            }
        },
        storageConfiguration = {
            "type": "OPENSEARCH_SERVERLESS",
            "opensearchServerlessConfiguration":opensearchServerlessConfiguration
        }
    )
    return create_kb_response["knowledgeBase"]

In [17]:
try:
    kb = create_knowledge_base_func()
except Exception as err:
    print(f"{err=}, {type(err)=}")

In [18]:
# Get KnowledgeBase 
get_kb_response = bedrock_agent_client.get_knowledge_base(knowledgeBaseId = kb['knowledgeBaseId'])

In [19]:
# Create a DataSource in KnowledgeBase 
create_ds_response = bedrock_agent_client.create_data_source(
    name = name,
    description = description,
    knowledgeBaseId = kb['knowledgeBaseId'],
    dataSourceConfiguration = {
        "type": "S3",
        "s3Configuration":s3Configuration
    },
    vectorIngestionConfiguration = {
        "chunkingConfiguration": chunkingStrategyConfiguration
    }
)
ds = create_ds_response["dataSource"]
print(ds)

{'knowledgeBaseId': 'ES25YKLMDG', 'dataSourceId': 'PHGYIKP84F', 'name': 'hackathon-bedrock-sample-knowledge-base', 'status': 'AVAILABLE', 'description': 'Banta Claws', 'dataSourceConfiguration': {'type': 'S3', 's3Configuration': {'bucketArn': 'arn:aws:s3:::hackathon-vsn-bedrock-kb'}}, 'vectorIngestionConfiguration': {'chunkingConfiguration': {'chunkingStrategy': 'FIXED_SIZE', 'fixedSizeChunkingConfiguration': {'maxTokens': 512, 'overlapPercentage': 20}}}, 'createdAt': datetime.datetime(2024, 10, 6, 11, 10, 46, 551576, tzinfo=tzlocal()), 'updatedAt': datetime.datetime(2024, 10, 6, 11, 10, 46, 551576, tzinfo=tzlocal())}


In [20]:
# Get DataSource 
bedrock_agent_client.get_data_source(knowledgeBaseId = kb['knowledgeBaseId'], dataSourceId = ds["dataSourceId"])

{'ResponseMetadata': {'RequestId': 'df64bb5a-5de9-4642-9e7f-d3e2116533e9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 06 Oct 2024 11:10:51 GMT',
   'content-type': 'application/json',
   'content-length': '570',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'df64bb5a-5de9-4642-9e7f-d3e2116533e9',
   'x-amz-apigw-id': 'fOZ5WGCBoAMEgqA=',
   'x-amzn-trace-id': 'Root=1-6702703b-7d7a07537f036e5b56314fe8'},
  'RetryAttempts': 0},
 'dataSource': {'knowledgeBaseId': 'ES25YKLMDG',
  'dataSourceId': 'PHGYIKP84F',
  'name': 'hackathon-bedrock-sample-knowledge-base',
  'status': 'AVAILABLE',
  'description': 'Banta Claws',
  'dataSourceConfiguration': {'type': 'S3',
   's3Configuration': {'bucketArn': 'arn:aws:s3:::hackathon-vsn-bedrock-kb'}},
  'vectorIngestionConfiguration': {'chunkingConfiguration': {'chunkingStrategy': 'FIXED_SIZE',
    'fixedSizeChunkingConfiguration': {'maxTokens': 512,
     'overlapPercentage': 20}}},
  'createdAt': datetime.datetime(2024, 10, 6, 11, 

In [22]:
# Start an ingestion job
start_job_response = bedrock_agent_client.start_ingestion_job(knowledgeBaseId = kb['knowledgeBaseId'], dataSourceId = ds["dataSourceId"])
job = start_job_response["ingestionJob"]8a
print(job)

{'knowledgeBaseId': 'ES25YKLMDG', 'dataSourceId': 'PHGYIKP84F', 'ingestionJobId': '0XCHJOWX8J', 'status': 'STARTING', 'statistics': {'numberOfDocumentsScanned': 0, 'numberOfNewDocumentsIndexed': 0, 'numberOfModifiedDocumentsIndexed': 0, 'numberOfDocumentsDeleted': 0, 'numberOfDocumentsFailed': 0}, 'startedAt': datetime.datetime(2024, 10, 6, 11, 11, 4, 566654, tzinfo=tzlocal()), 'updatedAt': datetime.datetime(2024, 10, 6, 11, 11, 4, 566654, tzinfo=tzlocal())}


In [23]:
# Get job 
while(job['status']!='COMPLETE' ):
    get_job_response = bedrock_agent_client.get_ingestion_job(
      knowledgeBaseId = kb['knowledgeBaseId'],
        dataSourceId = ds["dataSourceId"],
        ingestionJobId = job["ingestionJobId"]
  )
    job = get_job_response["ingestionJob"]
    


print(job)

{'knowledgeBaseId': 'ES25YKLMDG', 'dataSourceId': 'PHGYIKP84F', 'ingestionJobId': '0XCHJOWX8J', 'status': 'COMPLETE', 'statistics': {'numberOfDocumentsScanned': 1, 'numberOfNewDocumentsIndexed': 0, 'numberOfModifiedDocumentsIndexed': 0, 'numberOfDocumentsDeleted': 0, 'numberOfDocumentsFailed': 0}, 'startedAt': datetime.datetime(2024, 10, 6, 11, 11, 4, 566654, tzinfo=tzlocal()), 'updatedAt': datetime.datetime(2024, 10, 6, 11, 11, 5, 580457, tzinfo=tzlocal())}


In [24]:
# Print the knowledge base Id in bedrock, that corresponds to the Opensearch index in the collection we created before, we will use it for the invocation later
kb_id = kb["knowledgeBaseId"]
print(kb_id)

ES25YKLMDG


## RetreiveAndGenerate API

In [25]:
import boto3
import pprint
from botocore.client import Config

pp = pprint.PrettyPrinter(indent=2)

bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime')
bedrock_agent_client = boto3.client("bedrock-agent-runtime",
                              config=bedrock_config)
boto3_session = boto3.session.Session()
region_name = boto3_session.region_name

model_id = "anthropic.claude-3-haiku-20240307-v1:0" # try with both claude 3 Haiku as well as claude 3 Sonnet. for claude 3 Sonnet - "anthropic.claude-3-sonnet-20240229-v1:0"
region_id = region_name # replace it with the region you're running sagemaker notebook

In [26]:
def retrieveAndGenerate(input, kbId, sessionId=None, model_id = "anthropic.claude-3-haiku-20240307-v1:0", region_id = "us-east-1"):
    model_arn = f'arn:aws:bedrock:{region_id}::foundation-model/{model_id}'
    if sessionId:
        return bedrock_agent_client.retrieve_and_generate(
            input={
                'text': input
            },
            retrieveAndGenerateConfiguration={
                'type': 'KNOWLEDGE_BASE',
                'knowledgeBaseConfiguration': {
                    'knowledgeBaseId': kbId,
                    'modelArn': model_arn
                }
            },
            sessionId=sessionId
        )
    else:
        return bedrock_agent_client.retrieve_and_generate(
            input={
                'text': input
            },
            retrieveAndGenerateConfiguration={
                'type': 'KNOWLEDGE_BASE',
                'knowledgeBaseConfiguration': {
                    'knowledgeBaseId': kbId,
                    'modelArn': model_arn
                }
            }
        )

In [27]:
query = "Who is Banta Claws?"
response = retrieveAndGenerate(query, kb_id,model_id=model_id,region_id=region_id)
generated_text = response['output']['text']
print(generated_text)

Banta Claws is a character that the narrator believed in as a child, who is described as bringing gifts to children on Christmas Eve. According to the search results, Banta Claws is the narrator's version of Santa Claus, who has a workshop at the North Pole where he and his elves make toys throughout the year. On Christmas Eve, Banta Claws is said to bring the gifts to children's homes, traditionally coming down the chimney to leave the gifts near the Christmas tree.


In [28]:
citations = response["citations"]
contexts = []
for citation in citations:
    retrievedReferences = citation["retrievedReferences"]
    for reference in retrievedReferences:
         contexts.append(reference["content"]["text"])

print(contexts)

['So, I decided I will place a Christmas tree this  year so I even I could get a gift from Banta. But there was one small problem, my home does not  have a chimney\uf04a . I went to my mom and asked “Amma, I want a chimney in my room” She was  confused and thought it was some new toy, so she asked me again “You want a WHAT?” I said  “Chimney, Banta Claws will come through it” She laughed and said “Beta, Banta can come through  the window also, he doesn’t need a chimney for that”. I agreed reluctantly.    Finally, the Christmas Eve had come, I decorated a Christmas tree with stars and lights, I had placed  some milk and cookies for Banta to have, and wrote a small note of what I wanted as a gift from  Banta! I placed the note inside a small sock and hanged the sock to the Christmas tree. Just like it  was all shown in the children’s books which I used to read, and I went to sleep.    On that night my mom had placed the gift which she had purchased earlier, along with the note in  the wo

In [29]:
query = "What did the Narrator do on Christmas eve?"
response = retrieveAndGenerate(query, kb_id,model_id=model_id,region_id=region_id)
generated_text = response['output']['text']
print(generated_text)

On Christmas Eve, the narrator decorated a Christmas tree with stars and lights, placed some milk and cookies for Banta (Santa Claus) to have, and wrote a small note of what he wanted as a gift from Banta. He placed the note inside a small sock and hung the sock on the Christmas tree.


In [30]:
query = "Elaborate your answer"
response = retrieveAndGenerate(query, kb_id,model_id=model_id,region_id=region_id)
generated_text = response['output']['text']
print(generated_text)

Based on the search results, the story describes a young boy named Karthik who woke up on Christmas morning and found a gift from "Banta Claws" (Santa Claus) in a wooden box in his room. Karthik had placed a Christmas tree and a note for Santa, hoping to receive a gift, even though his home did not have a chimney. Karthik's mother had secretly placed the gift and note in the box, not wanting to disappoint him.


In [33]:
def retrieve(query, kbId, numberOfResults=5):
    return bedrock_agent_client.retrieve(
        retrievalQuery= {
            'text': query
        },
        knowledgeBaseId=kbId,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': numberOfResults,
                #'overrideSearchType': "HYBRID", # optional
            }
        }
    )

In [34]:
query = "Who is Banta Claws?"
response = retrieve(query, kb_id, 5)
retrievalResults = response['retrievalResults']
pp.pprint(retrievalResults)

[ { 'content': { 'text': 'Speech3: The day I got a gift from Banta '
                         'Claws!        I still remember that day when I was '
                         'just 10 year old child, I woke up early in the '
                         'morning the day  after Christmas Eve. I immediately '
                         'rushed towards the corner of the room where I had '
                         'placed a  wooden box. I opened the box, inside it '
                         'was a gift from Banta Claws!    I jumped with joy by '
                         'just by the sight of it. Finally my wish came true. '
                         'Along with the gift there was  also a note which '
                         'read “Karthik, my elves told me you were a good boy '
                         'this year, here is a gift for  you. Merry Christmas! '
                         '– Banta”. I ran to the kitchen holding the gift, '
                         '“Amma! Amma! See I told you  Banta was 

In [35]:
# fetch context from the response
def get_contexts(retrievalResults):
    contexts = []
    for retrievedResult in retrievalResults: 
        contexts.append(retrievedResult['content']['text'])
    return contexts

In [36]:
contexts = get_contexts(retrievalResults)
print(contexts)

['Speech3: The day I got a gift from Banta Claws!        I still remember that day when I was just 10 year old child, I woke up early in the morning the day  after Christmas Eve. I immediately rushed towards the corner of the room where I had placed a  wooden box. I opened the box, inside it was a gift from Banta Claws!    I jumped with joy by just by the sight of it. Finally my wish came true. Along with the gift there was  also a note which read “Karthik, my elves told me you were a good boy this year, here is a gift for  you. Merry Christmas! – Banta”. I ran to the kitchen holding the gift, “Amma! Amma! See I told you  Banta was real! Look I got a gift from him!” Such a wonderful feeling it was.    I always wanted a gift from Banta, I had read a lot of stories about Banta Claws in children’s book  and in movies. He had a workshop in North Pole where Banta and his millions of elves made toys  throughout the year. When it was time for Christmas Banta used to bring all the toys for the

In [37]:
prompt = f"""
Human: You are a financial advisor AI system, and provides answers to questions by using fact based and statistical information when possible. 
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{contexts}
</context>

<question>
{query}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:"""

In [38]:
# payload with model paramters
messages=[{ "role":'user', "content":[{'type':'text','text': prompt.format(contexts, query)}]}]
sonnet_payload = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 512,
    "messages": messages,
    "temperature": 0.5,
    "top_p": 1
        }  )

In [39]:
modelId = 'anthropic.claude-3-sonnet-20240229-v1:0' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
response = bedrock_client.invoke_model(body=sonnet_payload, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())
response_text = response_body.get('content')[0]['text']

print(response_text)

Based on the given context, Banta Claws appears to be a reference to Santa Claus, the legendary figure who is said to bring gifts to well-behaved children on Christmas Eve. The context mentions stories and beliefs about Banta Claws (Santa Claus) having a workshop at the North Pole where he and his elves make toys throughout the year. It also talks about him delivering gifts by coming through chimneys at night while children are asleep, leaving gifts under the Christmas tree. So Banta Claws seems to be the Indian/desi way of referring to the popular Christmas folklore character Santa Claus.


In [40]:
from langchain_aws import ChatBedrock
from langchain.retrievers.bedrock import AmazonKnowledgeBasesRetriever

llm = ChatBedrock(model_id=modelId, 
                  client=bedrock_client)

In [45]:
query = "Who is Banta Claws?"
retriever = AmazonKnowledgeBasesRetriever(
        knowledge_base_id=kb_id,
        retrieval_config={"vectorSearchConfiguration": 
                          {"numberOfResults": 4,
                           #'overrideSearchType': "SEMANTIC", # optional
                           }
                          },
        # endpoint_url=endpoint_url,
        # region_name=region,
        # credentials_profile_name="<profile_name>",
    )
docs = retriever.get_relevant_documents(
        query=query
    )
print(docs)

[Document(metadata={'location': {'type': 'S3', 's3Location': {'uri': 's3://hackathon-vsn-bedrock-kb/Hackathon LLM training.pdf'}}, 'score': 0.48894727}, page_content='Speech3: The day I got a gift from Banta Claws!        I still remember that day when I was just 10 year old child, I woke up early in the morning the day  after Christmas Eve. I immediately rushed towards the corner of the room where I had placed a  wooden box. I opened the box, inside it was a gift from Banta Claws!    I jumped with joy by just by the sight of it. Finally my wish came true. Along with the gift there was  also a note which read “Karthik, my elves told me you were a good boy this year, here is a gift for  you. Merry Christmas! – Banta”. I ran to the kitchen holding the gift, “Amma! Amma! See I told you  Banta was real! Look I got a gift from him!” Such a wonderful feeling it was.    I always wanted a gift from Banta, I had read a lot of stories about Banta Claws in children’s book  and in movies. He had a

In [46]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [47]:
response = rag_chain.invoke({"input": query})
pp.pprint(response["answer"])

('Based on the context provided, Banta Claws seems to be the Indian equivalent '
 'or version of Santa Claus. The passage describes Banta Claws as a figure who '
 'brings gifts to children on Christmas Eve, lives at the North Pole with '
 'elves who make toys, travels on a sleigh pulled by reindeers, and enters '
 'houses through chimneys to leave gifts. This closely mirrors the traditional '
 'Santa Claus story, but with the name adapted to "Banta Claws" in this Indian '
 'context.')


In [48]:
query = "What differences do you observe between Banta Claws and the folklore character Santa Claus?"
response = rag_chain.invoke({"input": query})
pp.pprint(response["answer"])

('Based on the context provided, there are a few key differences observed '
 'between Banta Claws and the traditional Santa Claus folklore character:\n'
 '\n'
 '1) Name: Banta Claws is the name used instead of Santa Claus.\n'
 '\n'
 '2) Entry method: While Santa is said to enter houses through the chimney, '
 'the mother mentions that Banta can come through the window as well since '
 "their home doesn't have a chimney.\n"
 '\n'
 '3) Cultural context: The story seems to be set in an Indian cultural '
 'context, whereas the Santa Claus tales originate from Western/Christian '
 'traditions.\n'
 '\n'
 'Overall, Banta Claws appears to be an Indian/local variation or adaptation '
 'of the popular Santa Claus character and Christmas traditions to fit the '
 'cultural context. But the core idea of a gift-giver for children remains the '
 'same.')
